## T Govardhan
### 9963250247, tgtgovardhan@gmail.com 

In [2]:
# importing neccessary libraries
import pandas as pd
import numpy as np

In [3]:
# loading the data
df = pd.read_csv('empd.csv')

In [4]:
# to see the  columns and structure of the data set(head() shows first 5 rows)
df.head()

,employee_id,first_name,last_name,email,phone,department,job_title,salary,join_date,performance_score
0,EMP0001,Robert,Vasquez,anthonyhubbard@example.com,395-797-9647,HR,Senior Analyst,89934.0,2022-09-06,5.0
1,EMP0002,Barbara,Bruce,bonniethomas@example.com,001-745-858-7716x94722,HR,Senior Consultant,77234.0,2024-07-05,5.0
2,EMP0003,Derek,Martin,smiller@example.net,356-450-4481x832,Enginering,Senior Consultant,92953.0,2022-11-02,3.0
3,EMP0004,Monique,Serrano,joseph51@example.net,001-852-702-8457,Sales,Senior Engineer,110460.0,2024-09-21,NaN
4,EMP0005,Frank,Robinson,mariepope@example.net,+1-788-681-6711x207,HR,Manager Manager,75316.0,2023-07-29,2.0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1260 entries, 0 to 1259
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   employee_id        1260 non-null   object 
 1   first_name         1100 non-null   object 
 2   last_name          1208 non-null   object 
 3   email              1149 non-null   object 
 4   phone              1260 non-null   object 
 5   department         1260 non-null   object 
 6   job_title          1260 non-null   object 
 7   salary             1125 non-null   float64
 8   join_date          1150 non-null   object 
 9   performance_score  1133 non-null   float64
dtypes: float64(2), object(8)
memory usage: 98.6+ KB


In [6]:
df.describe()

,salary,performance_score
count,1125.000000,1133.000000
mean,80578.008000,2.984996
std,19560.159354,1.394951
min,15174.000000,1.000000
25%,67488.000000,2.000000
50%,80939.000000,3.000000
75%,93046.000000,4.000000
max,157054.000000,5.000000


### 1) Missing Salary Imputation:

In [8]:
# Impute missing salary values using the median salary for each department
df['salary'] = df.groupby('department')['salary'].transform(lambda x: x.fillna(x.median()))

# Check if missing values still exist in salary column
df['salary'].isnull().sum()

0

### 2) Duplicate Employee ID Check:

In [10]:
# Identify and list all employee_id values that appear more than once

# Find duplicate employee IDs
duplicate_ids = df[df.duplicated(subset=['employee_id'], keep=False)]['employee_id'].unique()

# Print duplicate employee IDs
print("Duplicate Employee IDs:", '\n', duplicate_ids)

Duplicate Employee IDs: 
 ['EMP0018' 'EMP0037' 'EMP0052' 'EMP0054' 'EMP0064' 'EMP0096' 'EMP0103'
 'EMP0135' 'EMP0145' 'EMP0150' 'EMP0191' 'EMP0193' 'EMP0195' 'EMP0215'
 'EMP0247' 'EMP0257' 'EMP0265' 'EMP0267' 'EMP0271' 'EMP0275' 'EMP0292'
 'EMP0306' 'EMP0332' 'EMP0346' 'EMP0347' 'EMP0379' 'EMP0396' 'EMP0417'
 'EMP0438' 'EMP0458' 'EMP0468' 'EMP0478' 'EMP0487' 'EMP0492' 'EMP0567'
 'EMP0579' 'EMP0582' 'EMP0586' 'EMP0595' 'EMP0635' 'EMP0699' 'EMP0706'
 'EMP0718' 'EMP0745' 'EMP0750' 'EMP0760' 'EMP0802' 'EMP0824' 'EMP0878'
 'EMP0888' 'EMP0903' 'EMP0918' 'EMP0979' 'EMP1029' 'EMP1037' 'EMP1055'
 'EMP1061' 'EMP1076' 'EMP1091' 'EMP1126']


### 3) Performance Score Conversion:

In [12]:
# Performance Score Conversion: Convert to integer, replace NaN or non-numeric values with -1
df['performance_score'] = pd.to_numeric(df['performance_score'], errors='coerce').fillna(-1).astype(int)

### 4) Date Format Standardization:

In [14]:
# Date Format Standardization: Convert join_date to YYYY-MM-DD format
df['join_date'] = pd.to_datetime(df['join_date'], errors='coerce').dt.strftime('%Y-%m-%d')
# Remove rows with invalid join_date
df = df.dropna(subset=['join_date'])
df['join_year'] = df['join_date'].str[:4].astype(int)
# Display the updated DataFrame
print(df[['join_date', 'join_year']])

       join_date  join_year
0     2022-09-06       2022
1     2024-07-05       2024
2     2022-11-02       2022
3     2024-09-21       2024
4     2023-07-29       2023
...          ...        ...
1255  2023-10-15       2023
1256  2022-05-22       2022
1257  2020-05-30       2020
1258  2024-03-13       2024
1259  2021-01-03       2021

[1134 rows x 2 columns]


### 5) Department Typo Correction:

In [16]:
# knowing the different unique department names in the dataset 
df.department.unique()
len(df.department.unique())

9

In [17]:
# Department Typo Correction: Fix known typos
department_corrections = {'Enginering': 'Engineering','Marketing': 'Marketting'}
df['department'] = df['department'].replace(department_corrections)

In [18]:
# Print the unique values to verify corrections
df.department.unique()
len(df.department.unique())

7

### 6) Salary Outlier Removal:

In [20]:
# Calculate mean and standard deviation of salary
mean_salary = df['salary'].mean()
std_salary = df['salary'].std()

# Define upper bound for outlier removal
upper_bound = mean_salary + (3 * std_salary)

# Remove outliers (only salaries greater than the upper bound)
df_filtered = df[df['salary'] <= upper_bound]

# Display results
print(f"Original dataset size: {df.shape[0]}")
print(f"Filtered dataset size: {df_filtered.shape[0]}")
print(f"Number of outliers removed: {df.shape[0] - df_filtered.shape[0]}")

Original dataset size: 1134
Filtered dataset size: 1132
Number of outliers removed: 2


### Calculating the mean and standard deviation <font color="red">before!</font> removing any outliers

In [22]:
df.salary.describe()

count      1134.000000
mean      80527.694444
std       18418.616407
min       15174.000000
25%       68903.750000
50%       80705.000000
75%       91722.250000
max      157054.000000
Name: salary, dtype: float64

### Calculating the mean and standard deviation <font color="red">after! </font> removing the outliers

In [24]:
df_filtered.salary.describe()

count      1132.000000
mean      80406.161219
std       18203.221104
min       15174.000000
25%       68877.250000
50%       80705.000000
75%       91575.250000
max      132647.000000
Name: salary, dtype: float64

### 7) Email Domain Extraction:

In [26]:
# Extract domain from email, handling missing or invalid values
df['email_domain'] = df['email'].apply(lambda x: x.split('@')[-1] if isinstance(x, str) and '@' in x else None)

# Display the updated DataFrame with email and extracted domain
print(df[['email', 'email_domain']])

                             email email_domain
0       anthonyhubbard@example.com  example.com
1         bonniethomas@example.com  example.com
2              smiller@example.net  example.net
3             joseph51@example.net  example.net
4            mariepope@example.net  example.net
...                            ...          ...
1255           afisher@example.com  example.com
1256       robertjones@example.com  example.com
1257  reynoldsmichelle@example.org  example.org
1258                           NaN         None
1259        lynchdiane@example.net  example.net

[1134 rows x 2 columns]


### 8) Salary Correlation Calculation:

In [28]:
#Salary Correlation Calculation: Compute Pearson correlation
correlation = df[['salary', 'join_year']].corr().loc['salary', 'join_year']
print("Salary-Join Year Correlation:", correlation)

Salary-Join Year Correlation: 0.012061828206889112


### 9) Average Salary Aggregation:

In [30]:
#Average Salary Aggregation: Compute pivot table
department_salary_pivot = df.pivot_table(index='department', columns='job_title', values='salary', aggfunc='mean')
print(department_salary_pivot)

job_title    Director Analyst  Director Consultant  Director Engineer  \
department                                                              
Engineering      85111.200000         88926.250000       79945.444444   
Finance          79695.714286         85475.666667       84891.555556   
HR               82754.636364         79135.285714       70093.857143   
IT               76903.363636         90364.400000       76453.857143   
Marketting       76517.428571         76465.500000       79329.181818   
Operations       87076.208333         80941.000000       94778.750000   
Sales            75914.571429         80046.083333       74522.750000   

job_title    Director Manager  Junior Analyst  Junior Consultant  \
department                                                         
Engineering      87945.000000    75531.500000       89315.000000   
Finance          90154.500000    80370.187500       90413.900000   
HR               78207.000000    77863.444444       84401.750000   
IT

### 10) Duplicate Row Removal:

In [32]:
# Duplicate Row Removal: Drop duplicate rows
df = df.drop_duplicates()
print("Duplicates removed. Remaining duplicates:", df.duplicated().sum())

Duplicates removed. Remaining duplicates: 0


In [33]:
# Save the cleaned data
df.to_csv("cleaned_empd.csv", index=False)